## Tutorial #1: Loading EEG Data

#### This tutorial focuses on loading EEG data in several different ways.  

In order to load EEG data, MNE package will be used throughout this tutorial. MNE is an open-source Python package for exploring, visualizing, and analyzing human neurophysiological data. 

EEG data can be kept in 3 different types: raw data, epoched data and evoked (averaged) data.

1. Raw Data: Continues data is stored in raw object in MNE. A raw object has 2D array structure as channels×time.
   
2. Epoched Data: Collection of time-locked trials. It is stored as events×channels×times.

3. Evoked Data: When epoched data is averaged out over trials, evoked data is produced. The output is time-locked to the event. It is stored as channels×times.

In [ ]:
# For elimiating warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import mne

### 1.1) Loading datasets available in MNE: EEGBCI motor imagery Dataset


EEGBI motor imagery dataset is available in MNE package. The dataset contains 64-channel EEG recordings from 109 subjects and 14 runs on each subject in EDF+ format. For more details please check https://www.nmr.mgh.harvard.edu/mne/stable/manual/datasets_index.html#eegbci-motor-imagery. 

In order to load an available dataset in MNE, subject id or a list of subject ids, a list of runs (tasks) and path to locate data should be provided as arguments to load_data() function. 

load_data() function returns a list of paths that the requested data files located. By using read_raw_edf() function read downloaded edf files and lastly concatenate the raw data in each edf file to have the final dataset with all the selected runs.

In [15]:
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf

#Define the parameters 
subject = 1  # use data from subject 1
runs = [6, 10, 14]  # use only hand and feet motor imagery runs

#Get data and locate in to given path
files = eegbci.load_data(subject, runs, '../datasets/')
#Read raw data files where each file contains a run
raws = [read_raw_edf(f, preload=True) for f in files]
#Combine all loaded runs
raw_obj = concatenate_raws(raws)

Extracting EDF parameters from /Users/scg/Desktop/AlgorithmsNeuroscience/datasets/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/scg/Desktop/AlgorithmsNeuroscience/datasets/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/scg/Desktop/AlgorithmsNeuroscience/datasets/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


All data structures such as raw and epochs in mne have an attribute named 'info' which provides measurement related information about given data.  

In [16]:
raw_obj.info

<Info | 16 non-empty fields
    bads : list | 0 items
    ch_names : list | Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., ...
    chs : list | 64 items (EEG: 64)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 80.0 Hz
    meas_date : tuple | 2009-08-12 16:15:00 GMT
    nchan : int | 64
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 160.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xplotter

As mentioned at the beginning of the tutorial, raw type stores the data in 2D array format as channels×samples. The data can be accessed as following:

In [17]:
raw_data = raw_obj.get_data()

# OR

raw_data = raw_obj._data

print("Number of channels: ", str(len(raw_data)))
print("Number of samples: ", str(len(raw_data[0])))

Number of channels:  64
Number of samples:  60000


##### Converting Raw data to Epoched data

For the conversion, events in the data should be extracted firstly. If the data is annotated, then the events can be extracted easily by event_from_annotations() function of MNE. It returns both events and event ids.  

In [19]:
#Extract events from raw data
events, event_ids = mne.events_from_annotations(raw_obj, event_id='auto')
event_ids

Used Annotations descriptions: ['T0', 'T1', 'T2']


{'T0': 1, 'T1': 2, 'T2': 3}

To be able to define epochs, a time interval should be selected. tmin and tmax represents the beginning and the end of selected time interval (in seconds). This selection may differ, depending on the experiment. If the reponse to the given stimuli is expected to happen quickly, then a narrow time interval may be needed and vice versa. For example if an experiment is being conducted by showing pictures, then the reponses can be observed quickly. However when imagination is required during experiment, then a larger time interval may be needed to capture the whole response. 

In [22]:
tmin, tmax = -1, 4  # define epochs around events (in s)
#event_ids = dict(hands=2, feet=3)  # map event IDs to tasks

epochs = mne.Epochs(raw_obj, events, event_ids, tmin - 0.5, tmax + 0.5, baseline=None, preload=True)

90 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 90 events and 961 original time points ...
3 bad epochs dropped


Accessing data of an epochs object is being done by the same ways as for raw objcets.

In [107]:
#Access to the data
data = epochs._data

n_events = len(data) # or len(epochs.events)
print("Number of events: " + str(n_events)) 

n_channels = len(data[0,:]) # or len(epochs.ch_names)
print("Number of channels: " + str(n_channels))

n_times = len(data[0,0,:]) # or len(epochs.times)
print("Number of time instances: " + str(n_times))

Number of events: 189
Number of channels: 60
Number of time instances: 301


### 1.2) Loading data that is not available in MNE

If a dataset apart from MNE datasets is preferred, it can be loaded directly from given path. 
read_epoch() function can be used for reading epoched data from file. 

In [4]:
## Loading Epoched data 

# The file name should end with'epo' to be able to load it via mne
data_file = '../datasets/817_1_PDDys_ODDBALL_Clean_curated-epo'

# Read the EEG epochs:
epochs = mne.read_epochs(data_file + '.fif')
print(len(epochs))

Reading ../datasets/817_1_PDDys_ODDBALL_Clean_curated-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
189 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
189


In [24]:
epochs.info

<Info | 19 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fz, F3, F7, FC5, FC1, C3, T7, CP5, ...
    chs : list | 60 items (EEG: 60)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 60 items (60 EEG)
    events : list | 0 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 250.0 Hz
    meas_date : NoneType | unspecified
    meas_id : dict | 4 items
    nchan : int | 60
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 500.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xp

Data can be accessed as in the following cell.

In [29]:
## Accessing to the data

# The data can be accessed via:
epochs_data_1 = epochs._data
#or 
epochs_data_2 = epochs.get_data()

#To check whether the two different ways returns the same data
if epochs_data_1.all() == epochs_data_2.all():
    print("Output: Same data!")
else:
    print("Output: Data is not the same!")
        

Output: Same data!


event_id attribute keeps names of all the conditions in given data and number of events in each condition

In [27]:
epochs.event_id

{'Standard': 201, 'Target': 200, 'Novel': 202}

Accessing to data with a specific event condition can be done as following:

In [30]:
novels = epochs['Novel']

In [31]:
tmin = epochs.tmin
tmax = epochs.tmax

print('Start time before the event' , tmin)
print('Stop time after the event' , tmax)

Start time before the event -0.1
Stop time after the event 0.5


##### Get evoked data from epoched data

Evoked Potential is produced by averaging the epochs. If the data is type of eeg, then evoked data corresponds to ERP (Event Related Potential).  

In [4]:
evoked = epochs.average()
evoked.info

<Info | 19 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fz, F3, F7, FC5, FC1, C3, T7, CP5, ...
    chs : list | 60 items (EEG: 60)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 60 items (60 EEG)
    events : list | 0 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 250.0 Hz
    meas_date : NoneType | unspecified
    meas_id : dict | 4 items
    nchan : int | 60
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 500.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xp

Number of epochs that are used for averaging is kept in nave attribute of evoked object.

In [33]:
evoked.nave

189

In [5]:
evoked_data = evoked.data
n_channels = len(evoked_data) # or len(evoked.ch_names)
print("Number of channels: " + str(n_channels))

n_times = len(evoked_data[0,:]) # or len(evoked.times)
print("Number of time instances: " + str(n_times))

Number of channels: 60
Number of time instances: 301


As expected, number of channels and number of time instances do not change when epoched data converted to evoked data since the averaging is done over trials.